In [1]:
import numpy as np
from scipy.linalg import null_space, lstsq

class GeneralizedReducedGradient:
    """
    A class to perform optimization using the Generalized Reduced Gradient (GRG) algorithm.
    """

    def __init__(self, objective, eq_constraints, ineq_constraints=None, bounds=None,
                 objective_grad=None, eq_constraints_jac=None, ineq_constraints_jac=None,
                 tol=1e-6, max_iter=100, verbose=True):
        """
        Initialize the GRG Algorithm.

        Args:
            objective (function): Objective function f(x) to minimize.
            eq_constraints (list of functions): Equality constraints g_i(x) == 0.
            ineq_constraints (list of functions, optional): Inequality constraints h_i(x) <= 0.
            bounds (list of tuples, optional): Variable bounds as (min, max) for each variable.
            objective_grad (function, optional): Gradient of the objective function.
            eq_constraints_jac (list of functions, optional): Gradients of equality constraints.
            ineq_constraints_jac (list of functions, optional): Gradients of inequality constraints.
            tol (float, optional): Tolerance for constraint satisfaction and convergence.
            max_iter (int, optional): Maximum number of iterations.
            verbose (bool, optional): If True, print debugging information. Defaults to True.
        """
        self.objective = objective
        self.eq_constraints = eq_constraints.copy()
        self.ineq_constraints = ineq_constraints.copy() if ineq_constraints else []
        self.original_bounds = bounds.copy() if bounds else []
        self.bounds = bounds.copy() if bounds else []
        self.tol = tol
        self.max_iter = max_iter
        self.verbose = verbose

        # Analytical gradients and Jacobians
        self.objective_grad = objective_grad
        self.eq_constraints_jac = eq_constraints_jac.copy() if eq_constraints_jac else []
        self.ineq_constraints_jac = ineq_constraints_jac.copy() if ineq_constraints_jac else []

        # Slack variable management
        self.slack_vars = {}  # Mapping from inequality constraint index to slack variable index

        # Scaling factors
        self.var_scale = None
        self.const_scale = None

        # Number of original variables
        self.num_original_vars = len(self.original_bounds) if self.original_bounds else 0

    def _print(self, *args, **kwargs):
        """
        Internal method to handle printing based on verbosity.

        Args:
            *args: Variable length argument list for print.
            **kwargs: Arbitrary keyword arguments for print.
        """
        if self.verbose:
            print(*args, **kwargs)

    def compute_gradient(self, func_grad, x, func=None, eps=1e-8):
        """
        Compute the gradient of a function at x.

        Args:
            func_grad (function or None): Analytical gradient function.
            x (np.array): The point at which to compute the gradient.
            func (function, optional): The function to compute the gradient for (used if func_grad is None).
            eps (float, optional): A small perturbation for finite differences.

        Returns:
            np.array: The gradient vector.
        """
        if func_grad:
            try:
                # Compute analytical gradient for original variables
                grad_original = np.array(func_grad(x[:self.num_original_vars]))
            except Exception as e:
                self._print(f"Analytical gradient computation error: {e}")
                if func:
                    # Compute numerical gradient if analytical fails
                    grad_original = self.compute_gradient(None, x[:self.num_original_vars], func=func, eps=eps)
                else:
                    # Default to zeros if no function provided
                    grad_original = np.zeros(self.num_original_vars)
        else:
            if func is None:
                raise ValueError("Function must be provided for numerical gradient computation.")
            # Compute numerical gradient for original variables
            grad_original = np.zeros(self.num_original_vars)
            for i in range(self.num_original_vars):
                x_forward = np.copy(x)
                x_backward = np.copy(x)
                x_forward[i] += eps
                x_backward[i] -= eps
                try:
                    grad_original[i] = (func(x_forward) - func(x_backward)) / (2 * eps)
                except Exception as e:
                    self._print(f"Gradient computation error at index {i}: {e}")
                    grad_original[i] = 0.0

        # Append zeros for slack variables (if any)
        if len(x) > self.num_original_vars:
            slack_grad = np.zeros(len(x) - self.num_original_vars)
            grad = np.concatenate([grad_original, slack_grad])
        else:
            grad = grad_original

        return grad

    def compute_jacobian(self, active_constraints, x):
        """
        Compute the Jacobian matrix for active constraints at x.

        Args:
            active_constraints (list of dicts): List of active constraints with details.
            x (np.array): The point at which to compute the Jacobian.

        Returns:
            np.array: The Jacobian matrix.
        """
        m = len(active_constraints)
        n = len(x)
        J = np.zeros((m, n))
        eq_jac_idx = 0  # Index for equality Jacobians

        self._print(f"Computing Jacobian for {m} active constraints and {n} variables.")

        for i, constraint in enumerate(active_constraints):
            if constraint['type'] == 'equality':
                if self.eq_constraints_jac:
                    # Use the corresponding Jacobian function
                    if eq_jac_idx < len(self.eq_constraints_jac):
                        J[i, :self.num_original_vars] = self.eq_constraints_jac[eq_jac_idx](x[:self.num_original_vars])
                        self._print(f"Constraint {i}: Equality Jacobian row: {J[i, :]}")
                        eq_jac_idx += 1
                    else:
                        # If Jacobian functions are fewer, fallback to numerical
                        self._print(f"Constraint {i}: Insufficient analytical Jacobians for equality constraints. Using numerical gradient.")
                        J[i, :self.num_original_vars] = self.compute_gradient(None, x, func=constraint['function'], eps=1e-8)[:self.num_original_vars]
                else:
                    # Compute numerical gradient if no analytical Jacobian provided
                    self._print(f"Constraint {i}: Equality Jacobian not provided. Using numerical gradient.")
                    J[i, :self.num_original_vars] = self.compute_gradient(None, x, func=constraint['function'], eps=1e-8)[:self.num_original_vars]
            elif constraint['type'] == 'inequality':
                ineq_idx = constraint['index']
                if self.ineq_constraints_jac:
                    if ineq_idx < len(self.ineq_constraints_jac):
                        J[i, :self.num_original_vars] = self.ineq_constraints_jac[ineq_idx](x[:self.num_original_vars])
                        self._print(f"Constraint {i}: Inequality Jacobian row (original vars): {J[i, :self.num_original_vars]}")
                    else:
                        self._print(f"Constraint {i}: Insufficient analytical Jacobians for inequality constraints. Using numerical gradient.")
                        J[i, :self.num_original_vars] = self.compute_gradient(None, x, func=constraint['function'], eps=1e-8)[:self.num_original_vars]
                else:
                    # Compute numerical gradient if no analytical Jacobian provided
                    self._print(f"Constraint {i}: Inequality Jacobian not provided. Using numerical gradient.")
                    J[i, :self.num_original_vars] = self.compute_gradient(None, x, func=constraint['function'], eps=1e-8)[:self.num_original_vars]

                # Set derivative with respect to slack variable to 1
                slack_idx = constraint['slack_var_idx']
                J[i, slack_idx] = 1.0
                self._print(f"Constraint {i}: Inequality Jacobian row after slack variable: {J[i, :]}")

        self._print(f"Final Jacobian matrix:\n{J}")
        return J

    def scale_constraints_and_variables(self, x, active_constraints):
        """
        Scale variables and constraints to improve numerical stability.

        Args:
            x (np.array): Current variable vector.
            active_constraints (list of dicts): List of active constraints with details.

        Sets:
            self.var_scale: Scaling factors for variables.
            self.const_scale: Scaling factors for constraints.
        """
        # Variable scaling: scale variables to have maximum absolute value of 1
        self.var_scale = np.ones_like(x)
        for i in range(len(x)):
            if x[i] != 0:
                self.var_scale[i] = 1.0 / max(abs(x[i]), 1.0)
            else:
                self.var_scale[i] = 1.0

        # Constraint scaling: scale constraints to have maximum absolute value of 1
        num_constraints = len(active_constraints)
        self.const_scale = np.ones(num_constraints)
        for i in range(num_constraints):
            # Estimate the scaling based on initial constraints evaluation
            val = abs(active_constraints[i]['function'](x))
            if val > 0:
                self.const_scale[i] = 1.0 / max(val, 1.0)
            else:
                self.const_scale[i] = 1.0

    def identify_active_constraints(self, x):
        """
        Identify active equality and inequality constraints at point x.

        Args:
            x (np.array): The current point.

        Returns:
            list of dicts: List of active constraints with details.
        """
        active_constraints = []

        # Add all equality constraints (they are always active)
        for eq in self.eq_constraints:
            active_constraints.append({'type': 'equality', 'function': eq, 'jacobian': None})

        # Add active inequality constraints (those at their bounds)
        for idx, ineq in enumerate(self.ineq_constraints):
            if np.abs(ineq(x)) <= self.tol:
                active_constraints.append({'type': 'inequality', 'function': ineq, 'jacobian': None, 'index': idx})

        # Print active constraints
        self._print(f"Identified {len(active_constraints)} active constraints:")
        for i, constraint in enumerate(active_constraints):
            if constraint['type'] == 'equality':
                self._print(f"  Constraint {i}: Equality")
            elif constraint['type'] == 'inequality':
                self._print(f"  Constraint {i}: Inequality (Index {constraint['index']})")

        return active_constraints

    def add_slack_variables(self, active_constraints, x):
        """
        Convert active inequality constraints to equality constraints by adding slack variables.

        Args:
            active_constraints (list of dicts): List of active constraints with details.
            x (np.array): Current variable vector.

        Returns:
            tuple: (updated active_constraints list with modified functions, updated x with slack variables)
        """
        for constraint in active_constraints:
            if constraint['type'] == 'inequality':
                ineq_idx = constraint['index']
                if ineq_idx not in self.slack_vars:
                    # Define a new slack variable for this active inequality
                    slack_idx = len(x)
                    self.slack_vars[ineq_idx] = slack_idx

                    # Extend bounds to include slack variable (s_i >= 0)
                    self.bounds.append((0, None))

                    # Initialize slack variable in x
                    x = np.append(x, 0.0)

                    self._print(f"Added slack variable s_{ineq_idx} at index {slack_idx} for inequality constraint {ineq_idx}.")

                else:
                    slack_idx = self.slack_vars[ineq_idx]
                    self._print(f"Using existing slack variable s_{ineq_idx} at index {slack_idx} for inequality constraint {ineq_idx}.")

                # Modify the constraint function to include the slack variable: h(x) + s = 0
                original_ineq = self.ineq_constraints[ineq_idx]
                def modified_constraint(x, h=original_ineq, s_idx=slack_idx):
                    return h(x[:self.num_original_vars]) + x[s_idx]

                # Update the constraint function in active_constraints
                constraint['function'] = modified_constraint
                constraint['slack_var_idx'] = slack_idx

                self._print(f"Modified inequality constraint {ineq_idx} to include slack variable s_{ineq_idx}.")

        return active_constraints, x

    def remove_slack_variables(self, x):
        """
        Remove slack variables from the variable vector and update bounds accordingly.

        Args:
            x (np.array): Current variable vector.

        Returns:
            np.array: Variable vector without slack variables.
        """
        if not self.slack_vars:
            return x

        # Sort slack indices in descending order to remove from the end first
        slack_indices = sorted(self.slack_vars.values(), reverse=True)
        for idx in slack_indices:
            self._print(f"Removing slack variable at index {idx}.")
            # Remove slack variable from x
            x = np.delete(x, idx)
            # Remove corresponding bound
            self.bounds.pop(idx)

        # Clear slack_vars mapping
        self.slack_vars = {}
        return x

    def partition_variables(self, J, x):
        """
        Partition variables into independent and dependent variables based on active constraints and bounds.

        Args:
            J (np.array): Jacobian matrix of active constraints.
            x (np.array): Current variable vector.

        Returns:
            tuple: (independent_indices, dependent_indices)
        """
        n = len(x)
        m = J.shape[0]

        # Identify variables at their bounds (considering scaling)
        independent_indices = []
        for i, (lower, upper) in enumerate(self.bounds):
            if lower is not None and np.abs(x[i] - lower) <= self.tol:
                independent_indices.append(i)
            elif upper is not None and np.abs(x[i] - upper) <= self.tol:
                independent_indices.append(i)

        self._print(f"Variables at bounds (independent): {independent_indices}")

        # Determine additional independent variables to satisfy the number of constraints
        additional_indep = m - len(independent_indices)
        if additional_indep > 0:
            # Select variables with largest absolute gradient not already in independent_indices
            grad = np.abs(J).sum(axis=0)
            sorted_indices = np.argsort(-grad)
            for i in sorted_indices:
                if i not in independent_indices:
                    independent_indices.append(i)
                    additional_indep -= 1
                    if additional_indep == 0:
                        break
            self._print(f"Additional independent variables selected based on Jacobian gradients: {independent_indices[-additional_indep:] if additional_indep>0 else independent_indices}")

        # Ensure the number of independent variables matches the number of constraints
        if len(independent_indices) < m:
            # Add more variables if necessary
            for i in range(n):
                if i not in independent_indices:
                    independent_indices.append(i)
                    if len(independent_indices) == m:
                        break
            self._print(f"Final independent variables after ensuring count: {independent_indices}")

        # Dependent variables are those not in independent_indices
        dependent_indices = [i for i in range(n) if i not in independent_indices]
        self._print(f"Dependent Variables: {dependent_indices}")

        return independent_indices, dependent_indices

    def compute_reduced_gradient(self, grad, J):
        """
        Compute the reduced gradient.

        Args:
            grad (np.array): Gradient of the objective function.
            J (np.array): Jacobian matrix of active constraints.

        Returns:
            np.array: Reduced gradient.
        """
        if J.size == 0:
            return grad

        # Solve J^T * lambda = -grad for lambda using least squares
        JT = J.T
        try:
            lambda_, residuals, rank, s = lstsq(JT, -grad, lapack_driver='gelsy')
            self._print(f"Lagrange multipliers (lambda): {lambda_}")
        except np.linalg.LinAlgError:
            self._print("Singular Jacobian encountered while solving for lambda.")
            lambda_ = np.zeros(JT.shape[1])

        # Compute reduced gradient: grad + J^T * lambda
        reduced_grad = grad + JT @ lambda_

        self._print(f"Reduced Gradient: {reduced_grad}")
        return reduced_grad

    def compute_null_space(self, J):
        """
        Compute the null space of the Jacobian matrix J.

        Args:
            J (np.array): Jacobian matrix.

        Returns:
            np.array: Null space of J.
        """
        if J.size == 0:
            return np.identity(len(self.bounds))  # All variables are free

        null_sp = null_space(J)
        self._print(f"Computed null space with shape {null_sp.shape}:\n{null_sp}")
        return null_sp

    def find_search_direction(self, reduced_grad, null_space_J):
        """
        Calculate a search direction based on the reduced gradient and null space.

        Args:
            reduced_grad (np.array): Reduced gradient.
            null_space_J (np.array): Null space matrix.

        Returns:
            np.array: Search direction.
        """
        if np.linalg.norm(reduced_grad) < self.tol:
            self._print("Reduced gradient norm is below tolerance. No search direction needed.")
            return np.zeros_like(reduced_grad)

        N = null_space_J
        if N.size == 0:
            self._print("Null space is empty. No feasible directions available.")
            return np.zeros_like(reduced_grad)

        # Find c that minimizes ||reduced_grad + N c||^2
        # Solution: c = - (N^T N)^-1 N^T reduced_grad
        NtN = N.T @ N
        Nt_grad = N.T @ reduced_grad

        # Regularize NtN to improve numerical stability
        reg = 1e-8 * np.eye(NtN.shape[0])
        try:
            c = -np.linalg.solve(NtN + reg, Nt_grad)
            self._print(f"Coefficient vector c for search direction: {c}")
        except np.linalg.LinAlgError:
            self._print("Singular matrix encountered while solving for c. Using least squares solution.")
            c, residuals, rank, s = lstsq(NtN + reg, -Nt_grad, lapack_driver='gelsy')

        search_direction = N @ c
        self._print(f"Search Direction: {search_direction}")
        return search_direction

    def line_search_step(self, x, direction, independent_indices, dependent_indices, active_constraints):
        """
        Perform a line search to find the optimal step size using Wolfe conditions.

        Args:
            x (np.array): Current point.
            direction (np.array): Search direction.
            independent_indices (list): Indices of independent variables.
            dependent_indices (list): Indices of dependent variables.
            active_constraints (list of dicts): List of active constraints with details.

        Returns:
            float: Optimal step size alpha.
            str: Termination condition.
        """
        alpha = 1.0  # Initial step size
        alpha_min = 1e-8
        rho = 0.5     # Step size reduction factor
        c1 = 1e-4     # Armijo condition parameter
        c2 = 0.9      # Curvature condition parameter

        f_x = self.objective(x)
        grad_f = self.compute_gradient(self.objective_grad, x, func=self.objective)

        # Directional derivative
        grad_dir = np.dot(grad_f, direction)

        self._print(f"Initial objective value: {f_x}")
        self._print(f"Directional derivative: {grad_dir}")

        if grad_dir >= 0:
            self._print("Not a descent direction.")
            return 0, "not_descent_direction"

        while alpha > alpha_min:
            x_trial = np.copy(x)
            x_trial[independent_indices] += alpha * direction[independent_indices]
            self._print(f"Trial step with alpha={alpha}: {x_trial}")

            # Adjust dependent variables to satisfy constraints
            x_trial = self.newton_raphson_adjustment(x_trial, active_constraints, dependent_indices)

            f_trial = self.objective(x_trial)
            grad_f_trial = self.compute_gradient(self.objective_grad, x_trial, func=self.objective)
            grad_dir_trial = np.dot(grad_f_trial, direction)

            self._print(f"Trial objective value: {f_trial}")
            self._print(f"Trial directional derivative: {grad_dir_trial}")

            # Check Armijo condition
            if f_trial <= f_x + c1 * alpha * grad_dir:
                # Check curvature condition
                if grad_dir_trial >= c2 * grad_dir:
                    # Wolfe conditions satisfied
                    if self.is_feasible(x_trial, active_constraints):
                        self._print(f"Wolfe conditions satisfied with alpha={alpha}.")
                        return alpha, "wolfe_conditions_met"

            # Reduce step size
            alpha *= rho
            self._print(f"Reducing step size to alpha={alpha}.")

        self._print("Line search terminated without satisfying Wolfe conditions.")
        return alpha, "step_size_min"

    def is_feasible(self, x, active_constraints):
        """
        Check if x satisfies all active constraints and variable bounds.

        Args:
            x (np.array): Point to check.
            active_constraints (list of dicts): List of active constraints with details.

        Returns:
            bool: True if feasible, False otherwise.
        """
        # Check equality and inequality constraints
        for constraint in active_constraints:
            if np.abs(constraint['function'](x)) > self.tol:
                self._print(f"Constraint violated: {constraint['function'](x)} > tol.")
                return False

        # Check variable bounds
        for i, (lower, upper) in enumerate(self.bounds):
            if lower is not None and x[i] < lower - self.tol:
                self._print(f"Variable x[{i}] = {x[i]} is below lower bound {lower}.")
                return False
            if upper is not None and x[i] > upper + self.tol:
                self._print(f"Variable x[{i}] = {x[i]} is above upper bound {upper}.")
                return False

        self._print("All constraints and variable bounds are satisfied.")
        return True

    def newton_raphson_adjustment(self, x, active_constraints, dependent_indices, max_nr_iter=50):
        """
        Adjust dependent variables using Newton-Raphson to satisfy active constraints.

        Args:
            x (np.array): Current point.
            active_constraints (list of dicts): List of active constraints with details.
            dependent_indices (list): Indices of dependent variables.
            max_nr_iter (int, optional): Maximum Newton-Raphson iterations.

        Returns:
            np.array: Adjusted point.
        """
        x_new = np.copy(x)
        all_active = active_constraints

        for iteration in range(max_nr_iter):
            J = self.compute_jacobian(all_active, x_new)
            residuals = np.array([constraint['function'](x_new) for constraint in all_active])

            self._print(f"  Newton-Raphson Iteration {iteration+1}: Residuals = {residuals}")

            if np.linalg.norm(residuals) < self.tol:
                self._print("  Convergence achieved in Newton-Raphson adjustment.")
                break

            if J.size == 0 or not dependent_indices:
                self._print("  No constraints to satisfy or no dependent variables to adjust.")
                break  # No constraints to satisfy or no dependent variables to adjust

            # Extract Jacobian columns corresponding to dependent variables
            J_dep = J[:, dependent_indices]

            try:
                # Solve J_dep * delta = -residuals
                delta_dep, residuals_lsq, rank, s = lstsq(J_dep, -residuals, lapack_driver='gelsy')
                x_new[dependent_indices] += delta_dep

                self._print(f"  Adjusted dependent variables with delta: {delta_dep}")
                self._print(f"  Updated x: {x_new}")

                # Enforce variable bounds after each update
                for i in dependent_indices:
                    lower, upper = self.bounds[i]
                    if lower is not None and x_new[i] < lower:
                        x_new[i] = lower
                        self._print(f"  Variable x[{i}] adjusted to lower bound: {x_new[i]}")
                    if upper is not None and x_new[i] > upper:
                        x_new[i] = upper
                        self._print(f"  Variable x[{i}] adjusted to upper bound: {x_new[i]}")

                # Check for convergence
                if np.linalg.norm(delta_dep) < self.tol:
                    self._print("  Convergence achieved in Newton-Raphson adjustment.")
                    break
            except np.linalg.LinAlgError:
                self._print("  Singular Jacobian encountered during Newton-Raphson adjustment.")
                break

        return x_new

    def minimize(self, x0):
        """
        Perform the GRG optimization.

        Args:
            x0 (list or np.array): Initial guess for the variables.

        Returns:
            np.array: Optimal solution found by the GRG algorithm (original variables only).
        """
        x = np.array(x0, dtype=float)
        x = self.remove_slack_variables(x)  # Ensure no residual slack variables from previous runs

        for iteration in range(1, self.max_iter + 1):
            self._print(f"\n--- Iteration {iteration} ---")
            self._print(f"Current x: {x}")

            # Step 1: Identify active constraints at current x
            active_constraints = self.identify_active_constraints(x)
            active_eq = [c for c in active_constraints if c['type'] == 'equality']
            active_ineq = [c for c in active_constraints if c['type'] == 'inequality']
            self._print(f"Active Equality Constraints: {len(active_eq)}")
            self._print(f"Active Inequality Constraints: {len(active_ineq)}")

            # Step 2: Add slack variables for active inequality constraints
            active_constraints, x = self.add_slack_variables(active_constraints, x)
            self._print(f"Total Active Constraints (including slack): {len(active_constraints)}")

            # Step 3: Scale constraints and variables
            self.scale_constraints_and_variables(x, active_constraints)

            # Step 4: Compute Jacobian of active constraints
            J = self.compute_jacobian(active_constraints, x)

            # Step 5: Partition variables into independent and dependent
            independent_indices, dependent_indices = self.partition_variables(J, x)

            # Step 6: Compute gradient and reduced gradient
            grad = self.compute_gradient(self.objective_grad, x, func=self.objective)
            self._print(f"Gradient: {grad}")

            reduced_grad = self.compute_reduced_gradient(grad, J)
            norm_reduced_grad = np.linalg.norm(reduced_grad)
            self._print(f"||Reduced Gradient||: {norm_reduced_grad}")

            # Check convergence based on the reduced gradient norm
            if norm_reduced_grad < self.tol:
                self._print("Convergence achieved based on reduced gradient norm.")
                break

            # Step 7: Compute null space of J for feasible directions
            null_space_J = self.compute_null_space(J)

            if null_space_J.size == 0:
                self._print("No feasible directions available. Constraints may be too restrictive.")
                break

            # Step 8: Find the search direction
            search_direction = self.find_search_direction(reduced_grad, null_space_J)

            if np.linalg.norm(search_direction) < self.tol:
                self._print("Search direction is negligible. Optimization may have converged.")
                break

            # Step 9: Perform line search to find optimal step size
            alpha, termination = self.line_search_step(x, search_direction, independent_indices, dependent_indices, active_constraints)

            if termination in ["step_size_min", "not_descent_direction"]:
                self._print("Line search terminated without satisfying Wolfe conditions.")
                break

            # Step 10: Update variables with the step
            x_new = np.copy(x)
            x_new[independent_indices] += alpha * search_direction[independent_indices]
            self._print(f"x_new after independent step: {x_new}")

            # Step 11: Adjust dependent variables to satisfy constraints using Newton-Raphson
            x_new = self.newton_raphson_adjustment(x_new, active_constraints, dependent_indices)
            self._print(f"x_new after Newton-Raphson adjustment: {x_new}")

            # Print the objective function value
            f_x = self.objective(x)
            self._print(f"Objective Function Value: {f_x}")

            # Prepare for next iteration
            x = x_new

            # Step 12: Remove slack variables for the next iteration
            x = self.remove_slack_variables(x)
            self._print(f"x after removing slack variables: {x}")

        else:
            self._print("Maximum iterations reached without convergence.")

        # Final removal of any remaining slack variables before returning
        x = self.remove_slack_variables(x)
        self._print(f"Final x after ensuring removal of all slack variables: {x}")

        return x

In [2]:
# Example problem implementation with analytical gradients and Jacobians
def objective(x):
    """
    Objective function to minimize: f(x) = x1^2 + x2^2
    """
    return x[0]**2 + x[1]**2

def objective_grad(x):
    """
    Analytical gradient of the objective function: grad_f = [2*x1, 2*x2]
    """
    return np.array([2 * x[0], 2 * x[1]])

def eq_constraint1(x):
    """
    Equality constraint: x1 + x2 = 1
    """
    return x[0] + x[1] - 1

def eq_constraint1_jac(x):
    """
    Analytical Jacobian of the equality constraint: [1, 1]
    """
    return np.array([1.0, 1.0])

def ineq_constraint1(x):
    """
    Inequality constraint: x1 <= 0.5
    """
    return x[0] - 0.5

def ineq_constraint1_jac(x):
    """
    Analytical Jacobian of the inequality constraint: [1, 0]
    """
    return np.array([1.0, 0.0])

def ineq_constraint2(x):
    """
    Inequality constraint: x1 >= 0, which is equivalent to -x1 <= 0
    """
    return -x[0]

def ineq_constraint2_jac(x):
    """
    Analytical Jacobian of the inequality constraint: [-1, 0]
    """
    return np.array([-1.0, 0.0])

In [3]:
if __name__ == "__main__":
    # Define variable bounds
    # x1 >= 0, x1 <= 0.5, x2 is free
    bounds = [(0.0, 0.5), (None, None)]  # x1 bounds and x2 is unbounded

    # Initialize GRG Solver with analytical gradients and Jacobians
    grg_solver = GeneralizedReducedGradient(
        objective=objective,
        eq_constraints=[eq_constraint1],
        ineq_constraints=[ineq_constraint1, ineq_constraint2],
        bounds=bounds,
        objective_grad=objective_grad,
        eq_constraints_jac=[eq_constraint1_jac],
        ineq_constraints_jac=[ineq_constraint1_jac, ineq_constraint2_jac],
        tol=1e-6,
        max_iter=100,
        verbose=True
    )

    # Initial point
    x0 = [0.13, 0.81]

    # Solve the optimization problem
    solution = grg_solver.minimize(x0)
    print("\nOptimal solution:", solution)
    print("Objective value at optimum:", objective(solution))


--- Iteration 1 ---
Current x: [0.13 0.81]
Identified 1 active constraints:
  Constraint 0: Equality
Active Equality Constraints: 1
Active Inequality Constraints: 0
Total Active Constraints (including slack): 1
Computing Jacobian for 1 active constraints and 2 variables.
Constraint 0: Equality Jacobian row: [1. 1.]
Final Jacobian matrix:
[[1. 1.]]
Variables at bounds (independent): []
Additional independent variables selected based on Jacobian gradients: [0]
Dependent Variables: [1]
Gradient: [0.26 1.62]
Lagrange multipliers (lambda): [-0.94]
Reduced Gradient: [-0.68  0.68]
||Reduced Gradient||: 0.9616652224137047
Computed null space with shape (2, 1):
[[-0.70710678]
 [ 0.70710678]]
Coefficient vector c for search direction: [-0.96166521]
Search Direction: [ 0.67999999 -0.67999999]
Initial objective value: 0.6730000000000002
Directional derivative: -0.9247999907520001
Trial step with alpha=1.0: [0.80999999 0.81      ]
Computing Jacobian for 1 active constraints and 2 variables.
Constr